In [8]:
push!(LOAD_PATH,".")
push!(LOAD_PATH,"../..")

10-element Vector{String}:
 "/airlab"
 "/airlab/src/mdma_greedy"
 "/env"
 "@"
 "@v#.#"
 "@stdlib"
 "."
 "../.."
 "."
 "../.."

In [9]:
using Revise
using POMDPs
using DiscreteValueIteration
using POMDPTools
# using Dates
# using Profile

In [10]:
using MDMA

┌ Info: Precompiling MDMA [top-level]
└ @ Base loading.jl:1664


Test Summary: | Pass  Total  Time
detectTarget  |   30     30  0.4s
Test Summary:        |Time
single_robot_planner | None  0.0s
Test Summary:   |Time
test_file_parse | None  0.0s


In [11]:
# Set model parameters
sensor = ViewConeSensor(pi/2, 3);
horizon = 60
grid = MDMA_Grid(25,25, horizon);

# Setup initial distribution of targets
targets = Vector{Target}(undef, 0);
push!(targets, Target(grid.width-3,grid.height-3,0,1));
push!(targets, Target(grid.width-4,grid.height-2,0,2));
push!(targets, Target(grid.width-5,grid.height-3,0,3));
initial_state = MDPState(UAVState(1, 1, :S), horizon)
num_robots = 1

target_trajectories = MDMA.generate_target_trajectories(grid, horizon, targets);

move_dist = 3;
multi_problem = MultiRobotTargetCoverageProblem(Vector{MDPState}([initial_state]), target_trajectories, grid=grid, sensor=sensor, horizon=horizon, move_dist=move_dist, solver_iterations=100,belres=0.01)
coverage = generate_empty_coverage_data(multi_problem)
model = MDMA.SingleRobotMultiTargetViewCoverageProblem(grid, sensor, horizon, target_trajectories, move_dist, coverage, initial_state)

SingleRobotMultiTargetViewCoverageProblem(MDMA_Grid(25, 25, 8, 60, [MDPState(UAVState(1, 1, :E), 1, 60, nothing) MDPState(UAVState(2, 1, :E), 1, 60, nothing) … MDPState(UAVState(24, 1, :E), 1, 60, nothing) MDPState(UAVState(25, 1, :E), 1, 60, nothing); MDPState(UAVState(1, 2, :E), 1, 60, nothing) MDPState(UAVState(2, 2, :E), 1, 60, nothing) … MDPState(UAVState(24, 2, :E), 1, 60, nothing) MDPState(UAVState(25, 2, :E), 1, 60, nothing); … ; MDPState(UAVState(1, 24, :E), 1, 60, nothing) MDPState(UAVState(2, 24, :E), 1, 60, nothing) … MDPState(UAVState(24, 24, :E), 1, 60, nothing) MDPState(UAVState(25, 24, :E), 1, 60, nothing); MDPState(UAVState(1, 25, :E), 1, 60, nothing) MDPState(UAVState(2, 25, :E), 1, 60, nothing) … MDPState(UAVState(24, 25, :E), 1, 60, nothing) MDPState(UAVState(25, 25, :E), 1, 60, nothing);;; MDPState(UAVState(1, 1, :NE), 1, 60, nothing) MDPState(UAVState(2, 1, :NE), 1, 60, nothing) … MDPState(UAVState(24, 1, :NE), 1, 60, nothing) MDPState(UAVState(25, 1, :NE), 1, 60,

In [11]:
policy = MDMA.solve_single_robot(model)


[Iteration 1   ] residual:         10 | iteration runtime:  82295.730 ms, (      82.3 s total)
[Iteration 2   ] residual:          0 | iteration runtime:  83420.867 ms, (       166 s total)


ValueIterationPolicy:
 MDPState(UAVState(1, 1, :E), 1, 60, nothing) -> MDPState(UAVState(3, 1, :NE), 2, 60, nothing)
 MDPState(UAVState(1, 2, :E), 1, 60, nothing) -> MDPState(UAVState(3, 1, :NE), 2, 60, nothing)
 MDPState(UAVState(1, 3, :E), 1, 60, nothing) -> MDPState(UAVState(3, 1, :NE), 2, 60, nothing)
 MDPState(UAVState(1, 4, :E), 1, 60, nothing) -> MDPState(UAVState(3, 2, :NE), 2, 60, nothing)
 MDPState(UAVState(1, 5, :E), 1, 60, nothing) -> MDPState(UAVState(3, 3, :NE), 2, 60, nothing)
 MDPState(UAVState(1, 6, :E), 1, 60, nothing) -> MDPState(UAVState(3, 4, :NE), 2, 60, nothing)
 MDPState(UAVState(1, 7, :E), 1, 60, nothing) -> MDPState(UAVState(3, 5, :NE), 2, 60, nothing)
 MDPState(UAVState(1, 8, :E), 1, 60, nothing) -> MDPState(UAVState(3, 6, :NE), 2, 60, nothing)
 MDPState(UAVState(1, 9, :E), 1, 60, nothing) -> MDPState(UAVState(3, 7, :NE), 2, 60, nothing)
 MDPState(UAVState(1, 10, :E), 1, 60, nothing) -> MDPState(UAVState(3, 8, :NE), 2, 60, nothing)
 MDPState(UAVState(1, 11, :

In [12]:

path = compute_path(model, policy, MDPState(UAVState(1,1,:N), model.horizon))

60-element Vector{MDPState}:
 MDPState(UAVState(1, 1, :N), 1, 60, nothing)
 MDPState(UAVState(3, 1, :NW), 2, 60, nothing)
 MDPState(UAVState(5, 2, :W), 3, 60, nothing)
 MDPState(UAVState(7, 4, :SW), 4, 60, nothing)
 MDPState(UAVState(9, 6, :S), 5, 60, nothing)
 MDPState(UAVState(11, 8, :SE), 6, 60, nothing)
 MDPState(UAVState(13, 10, :E), 7, 60, nothing)
 MDPState(UAVState(15, 12, :NE), 8, 60, nothing)
 MDPState(UAVState(17, 14, :E), 9, 60, nothing)
 MDPState(UAVState(17, 13, :NE), 10, 60, nothing)
 MDPState(UAVState(17, 12, :E), 11, 60, nothing)
 MDPState(UAVState(17, 11, :NE), 12, 60, nothing)
 MDPState(UAVState(17, 10, :E), 13, 60, nothing)
 ⋮
 MDPState(UAVState(19, 22, :E), 49, 60, nothing)
 MDPState(UAVState(19, 22, :NE), 50, 60, nothing)
 MDPState(UAVState(19, 22, :E), 51, 60, nothing)
 MDPState(UAVState(19, 22, :NE), 52, 60, nothing)
 MDPState(UAVState(19, 22, :E), 53, 60, nothing)
 MDPState(UAVState(19, 22, :NE), 54, 60, nothing)
 MDPState(UAVState(19, 22, :E), 55, 60, nothing)

In [13]:
# Type signature of RenderConf for reference
#struct RenderConf
#    ppm::Float64 # Pixels per meter for grid size
#    buf::Int64   # Buffer grid surrounding the image
#    draw_paths::Bool # Render paths
#    draw_old_states::Bool # Set to false if you want only current state shown
#end

draw_frames(RenderConf(50, 4, true, false), model, path)

Writing to output/01.png
Writing to output/02.png
Writing to output/03.png
Writing to output/04.png
Writing to output/05.png
Writing to output/06.png
Writing to output/07.png
Writing to output/08.png
Writing to output/09.png
Writing to output/10.png
Writing to output/11.png
Writing to output/12.png
Writing to output/13.png
Writing to output/14.png
Writing to output/15.png
Writing to output/16.png
Writing to output/17.png
Writing to output/18.png
Writing to output/19.png
Writing to output/20.png
Writing to output/21.png
Writing to output/22.png
Writing to output/23.png
Writing to output/24.png
Writing to output/25.png
Writing to output/26.png
Writing to output/27.png
Writing to output/28.png
Writing to output/29.png
Writing to output/30.png
Writing to output/31.png
Writing to output/32.png
Writing to output/33.png
Writing to output/34.png
Writing to output/35.png
Writing to output/36.png
Writing to output/37.png
Writing to output/38.png
Writing to output/39.png
Writing to output/40.png
